### 〇，编程环境

以下过程为Mac系统上单机版Spark练习编程环境的配置方法。
注意：仅配置练习环境无需安装hadoop,无需安装scala.


1，安装Java8

https://www.oracle.com/technetwork/java/javase/downloads/jdk8-downloads-2133151.html

注意避免安装其它版本的jdk否则可能会有不兼容spark的情况


2，下载spark并解压

http://spark.apache.org/downloads.html

解压到以下路径：

Users/liangyun/ProgramFiles/spark-2.4.3-bin-hadoop2.7


3，配置spark环境

vim ~/.bashrc

插入下面两条语句

export SPARK_HOME=/Users/liangyun/ProgramFiles/spark-2.4.3-bin-hadoop2.7

export PATH=$PATH:$SPARK_HOME/bin


4，配置jupyter支持

若未有安装jupyter可以下载Anaconda安装之。
pip install toree

jupyter toree install --spark_home=your-spark-home


### 一，运行Spark

Spark主要通过以下一些方式运行。

1，通过spark-shell进入spark交互式环境,使用Scala语言。

2，通过spark-submit提交Spark应用程序进行批处理。
这种方式可以提交Scala或Java语言编写后生成的jar包或者Python脚本。

3，通过pyspark进入pyspark交互式环境，使用Python语言。
这种方式可以指定jupyter或者ipython为交互环境。

4，通过zepplin notebook交互式执行。
zepplin是jupyter notebook的apache对应产品。

5, 安装Apache Toree - Scala内核。
可以在jupyter 中运行spark-shell。

使用spark-shell运行时，还可以添加常用的两个参数。

一个是-master指定使用何种分布类型。

第二个是-jars指定依赖的jar包。

In [ ]:
//local本地模式运行，默认使用4个逻辑CPU内核
spark-shell

//local本地模式运行，使用全部内核，添加 code.jar到classpath
spark-shell  --master local[*] --jars code.jar 

//local本地模式运行，使用4个内核
spark-shell  --master local[4]

//standalone模式连接集群，指定url和端口号
spark-shell  --master spark://master:7077

//客户端模式连接YARN集群，Driver运行在本地，方便查看日志，调试时推荐使用。
spark-shell  --master yarn-client

//集群模式连接YARN集群，Driver运行在集群，本地机器计算和通信压力小，批量任务时推荐使用。
spark-shell  --master yarn-cluster



In [ ]:
//提交scala写的任务
./bin/spark-submit --class org.apache.spark.examples.SparkPi \
 --master yarn \
 --deploy-mode cluster \
 --driver-memory 4g \
 --executor-memory 2g \
 --executor-cores 1 \
 --queue thequeue \
 examples/jars/spark-examples*.jar 10 

In [ ]:
//提交python写的任务
spark-submit --master yarn \
--executor-memory 6G \
--driver-memory 6G \
--deploy-mode cluster \
--num-executors 600 \
--conf spark.yarn.maxAppAttempts=1 \
--executor-cores 1 \
--conf spark.default.parallelism=2000 \
--conf spark.task.maxFailures=10 \
--conf spark.stage.maxConsecutiveAttempts=10 \
test.py 

### 一，创建RDD

创建RDD主要有两种方式，一个是textFile加载本地或者集群文件系统中的数据，

第二个是用parallelize方法将Driver中的数据结构并行化成RDD。

In [1]:
//从本地文件系统中加载数据
val file = "file:///Users/liangyun/CodeFiles/spark_tutorial/data.txt"
val rdd = sc.textFile(file,3)
rdd.collect.foreach(println)

1
2
3
4
5


file = file:///Users/liangyun/CodeFiles/spark_tutorial/data.txt
rdd = file:///Users/liangyun/CodeFiles/spark_tutorial/data.txt MapPartitionsRDD[1] at textFile at <console>:29


file:///Users/liangyun/CodeFiles/spark_tutorial/data.txt MapPartitionsRDD[1] at textFile at <console>:29

(0,0)
(0,0)
(0,0)
(14,1)
(7,1)
(29,2)
(15,2)
(45,3)
(24,3)
(62,4)
(34,4)
(80,5)
(45,5)
(99,6)
(57,6)
(1,1)
(3,2)
(6,3)
(10,4)
(15,5)
((0,0),(70,7))
((70,7),(119,7))
((189,14),(21,6))


In [ ]:
//从集群文件系统中加载数据
val file = "hdfs://localhost:9000/user/hadoop/data.txt"
//可以省去hdfs://localhost:9000
val rdd = sc.textFile(file,3)

In [47]:
//parallelize将Driver中的数据结构生成RDD,第二个参数指定分区数
val rdd = sc.parallelize(1 to 10,2)
rdd.collect

rdd = ParallelCollectionRDD[55] at parallelize at <console>:30


Array(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

In [48]:
//makeRDD的作用和parallelize一样
val rdd = sc.makeRDD(1 to 10,2)
rdd.collect

rdd = ParallelCollectionRDD[56] at makeRDD at <console>:30


Array(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

### 二，常用Action操作

Action操作将触发基于RDD依赖关系的计算。

**collect**

In [1]:
val rdd = sc.parallelize(1 to 10,5) 

rdd = ParallelCollectionRDD[0] at parallelize at <console>:27


ParallelCollectionRDD[0] at parallelize at <console>:27

In [6]:
//collect操作将数据汇集到Driver,数据过大时有超内存风险
val all_data = rdd.collect()

all_data = Array(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)


Array(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

**take**

In [7]:
//take操作将前若干个数据汇集到Driver，相比collect安全
val rdd = sc.parallelize(1 to 10,5) 
val part_data = rdd.take(4)

rdd = ParallelCollectionRDD[5] at parallelize at <console>:30
part_data = Array(1, 2, 3, 4)


Array(1, 2, 3, 4)

**takeSample**

In [14]:
//takeSample可以随机取若干个到Driver,第一个参数设置是否放回抽样
val rdd = sc.parallelize(1 to 10,5) 
val sample_data = rdd.takeSample(false,10,0)

rdd = ParallelCollectionRDD[9] at parallelize at <console>:30
sample_data = Array(5, 9, 10, 7, 4, 6, 3, 2, 8, 1)


Array(5, 9, 10, 7, 4, 6, 3, 2, 8, 1)

**first**

In [15]:
//first取第一个数据
val rdd = sc.parallelize(1 to 10,5) 
val first_data = rdd.first

rdd = ParallelCollectionRDD[10] at parallelize at <console>:30
first_data = 1


1

**count**

In [17]:
//count查看RDD元素数量
val rdd = sc.parallelize(1 to 10,5)
val data_count = rdd.count

rdd = ParallelCollectionRDD[11] at parallelize at <console>:30
data_count = 10


10

**reduce**

In [18]:
//reduce利用二元函数对数据进行规约
val rdd = sc.parallelize(1 to 10,5) 
rdd.reduce(_+_)

rdd = ParallelCollectionRDD[12] at parallelize at <console>:30


55

**foreach**

In [27]:
//foreach对每一个元素执行某种操作，不生成新的RDD
//累加器用法详见共享变量
val rdd = sc.parallelize(1 to 10,5) 
var accum = sc.longAccumulator("sumAccum")
rdd.foreach(x => accum.add(x))
print(accum.value)

55

rdd = ParallelCollectionRDD[24] at parallelize at <console>:33
accum = LongAccumulator(id: 777, name: Some(sumAccum), value: 55)


LongAccumulator(id: 777, name: Some(sumAccum), value: 55)

**countByKey**

In [28]:
//countByKey对Pair RDD按key统计数量
val pairRdd = sc.parallelize(Array((1,1),(1,4),(3,9),(2,16))) 
pairRdd.countByKey

pairRdd = ParallelCollectionRDD[25] at parallelize at <console>:30


Map(1 -> 2, 2 -> 1, 3 -> 1)

**saveAsTextFile和saveAsObjectFile**

In [38]:
//saveAsTextFile保存rdd成text文件到本地,类似作用的
val file = "file:///Users/liangyun/CodeFiles/spark_tutorial/rdddata"
val rdd = sc.parallelize(1 to 5)
rdd.saveAsTextFile(file)

file = file:///Users/liangyun/CodeFiles/spark_tutorial/rdddata
rdd = ParallelCollectionRDD[42] at parallelize at <console>:32


ParallelCollectionRDD[42] at parallelize at <console>:32

In [39]:
//重新读入
val data = sc.textFile(file)
data.collect.foreach(println)

1
2
3
4
5


data = file:///Users/liangyun/CodeFiles/spark_tutorial/rdddata MapPartitionsRDD[45] at textFile at <console>:32


file:///Users/liangyun/CodeFiles/spark_tutorial/rdddata MapPartitionsRDD[45] at textFile at <console>:32

In [ ]:
//saveAsObjectFile保存rdd成Object文件，类似的还有saveAsSequenceFile
rdd3 = rdd.saveAsObjectFile("file:///Users/liangyun/CodeFiles/spark_tutorial/rdddata3")

### 三，常用Transformation操作

Transformation转换操作具有懒惰执行的特性，它只指定新的RDD和其父RDD的依赖关系，只有当Action操作触发到该依赖的时候，它才被计算。

**map**

In [40]:
//map操作对每个元素进行一个映射转换
val rdd = sc.parallelize(1 to 10,3)
rdd.collect

rdd = ParallelCollectionRDD[46] at parallelize at <console>:30


Array(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

In [41]:
rdd.map(_+1).collect

Array(2, 3, 4, 5, 6, 7, 8, 9, 10, 11)

**filter**

In [49]:
//filter应用过滤条件过滤掉一些数据
val rdd = sc.parallelize(1 to 10,3)
rdd.filter(_>5).collect

rdd = ParallelCollectionRDD[57] at parallelize at <console>:30


Array(6, 7, 8, 9, 10)

**flatMap**

In [43]:
//flatMap操作执行将每个元素生成一个Array后压平
val rdd = sc.parallelize(Array("hello world","hello China"))
rdd.map(_.split(" ")).collect

rdd = ParallelCollectionRDD[50] at parallelize at <console>:30


Array(Array(hello, world), Array(hello, China))

In [44]:
rdd.flatMap(_.split(" ")).collect

Array(hello, world, hello, China)

**sample**

In [50]:
//sample对原rdd在每个分区按照比例进行抽样,第一个参数设置是否可以重复抽样
val rdd = sc.parallelize(1 to 10,1)
rdd.sample(false,0.5,0).collect

rdd = ParallelCollectionRDD[59] at parallelize at <console>:30


Array(1, 2, 4, 8, 9, 10)

**distinct**

In [51]:
//distinct去重
val rdd = sc.parallelize(Array(1,1,2,2,3,3,4,5))
rdd.distinct.collect

rdd = ParallelCollectionRDD[61] at parallelize at <console>:30


Array(4, 1, 5, 2, 3)

**subtract**

In [52]:
//subtract找到属于前一个rdd而不属于后一个rdd的元素
val a = sc.parallelize(1 to 10)
val b = sc.parallelize(5 to 15)
a.subtract(b).collect

a = ParallelCollectionRDD[65] at parallelize at <console>:28
b = ParallelCollectionRDD[66] at parallelize at <console>:29


Array(4, 1, 2, 3)

**union** 

In [53]:
//union合并数据
val a = sc.parallelize(1 to 5)
val b = sc.parallelize(3 to 8)
a.union(b).collect

a = ParallelCollectionRDD[71] at parallelize at <console>:31
b = ParallelCollectionRDD[72] at parallelize at <console>:32


Array(1, 2, 3, 4, 5, 3, 4, 5, 6, 7, 8)

**intersection**

In [55]:
//intersection求交集
val a = sc.parallelize(1 to 5)
val b = sc.parallelize(3 to 8)
a.intersection(b).collect

a = ParallelCollectionRDD[80] at parallelize at <console>:31
b = ParallelCollectionRDD[81] at parallelize at <console>:32


Array(4, 5, 3)

**cartesian**

In [56]:
//cartesian笛卡尔积
val boys = sc.parallelize(Array("LiLei","Tom"))
val girls = sc.parallelize(Array("HanMeiMei","Lily"))
boys.cartesian(girls).collect


boys = ParallelCollectionRDD[88] at parallelize at <console>:28
girls = ParallelCollectionRDD[89] at parallelize at <console>:29


Array((LiLei,HanMeiMei), (LiLei,Lily), (Tom,HanMeiMei), (Tom,Lily))

**pipe**

In [59]:
//pipe可以调用外部可执行程序进行转换操作生成新的rdd，类似于hadoop streaming
val rdd = sc.parallelize(Array("hello","world","hi","apache"))

rdd = ParallelCollectionRDD[103] at parallelize at <console>:28


ParallelCollectionRDD[103] at parallelize at <console>:28

In [ ]:
//getlen.py
/*
import sys
for line in sys.stdin:
    print(str(len(line.rstrip)))

*/

In [61]:
val rdd2 = rdd.pipe("python getlen.py")
rdd2.collect

rdd2 = PipedRDD[104] at pipe at <console>:27


Array(5, 5, 2, 6)

**sortBy**

In [58]:
//按照某种方式进行排序
//指定按照第3个元素大小进行排序
val rdd = sc.parallelize(Array((1,2,3),(3,2,2),(4,1,1)))
rdd.sortBy(_._3).collect

rdd = ParallelCollectionRDD[97] at parallelize at <console>:31


Array((4,1,1), (3,2,2), (1,2,3))

### 四，常用PairRDD的转换操作

PairRDD指的是数据为Tuple2数据类型的RDD,其每个数据的第一个元素被当做key，第二个元素被当做value. 

**reduceByKey**

In [62]:
//reduceByKey对相同的key对应的values应用二元归并操作
val rdd = sc.parallelize(Array(("hello",1),("world",2),
                               ("hello",3),("world",5)))
rdd.reduceByKey(_+_).collect

rdd = ParallelCollectionRDD[105] at parallelize at <console>:30


Array((hello,4), (world,7))

**groupByKey**

In [63]:
//groupByKey将相同的key对应的values收集成一个Iterator
val rdd = sc.parallelize(Array(("hello",1),("world",2),
                               ("hello",3),("world",5)))
rdd.groupByKey().collect

rdd = ParallelCollectionRDD[107] at parallelize at <console>:30


Array((hello,CompactBuffer(1, 3)), (world,CompactBuffer(2, 5)))

**sortByKey**

In [66]:
//sortByKey按照key排序,可以指定是否降序
val rdd = sc.parallelize(Array(("hello",1),("world",2),
                               ("China",3),("Beijing",5)))
rdd.sortByKey(false).collect

rdd = ParallelCollectionRDD[117] at parallelize at <console>:30


Array((world,2), (hello,1), (China,3), (Beijing,5))

**join**

In [70]:
//join相当于根据key进行内连接
val age = sc.parallelize(Array(("LiLei",18),
                        ("HanMeiMei",16),("Jim",20)))
val gender = sc.parallelize(Array(("LiLei","male"),
                        ("HanMeiMei","female"),("Lucy","female")))
age.join(gender).collect

age = ParallelCollectionRDD[136] at parallelize at <console>:31
gender = ParallelCollectionRDD[137] at parallelize at <console>:33


Array((HanMeiMei,(16,female)), (LiLei,(18,male)))

**leftOuterJoin和rightOuterJoin**

In [71]:
//leftOuterJoin相当于关系表的左连接

val age = sc.parallelize(Array(("LiLei",18),
                        ("HanMeiMei",16)))
val gender = sc.parallelize(Array(("LiLei","male"),
                        ("HanMeiMei","female"),("Lucy","female")))
age.leftOuterJoin(gender).collect

age = ParallelCollectionRDD[141] at parallelize at <console>:32
gender = ParallelCollectionRDD[142] at parallelize at <console>:34


Array((HanMeiMei,(16,Some(female))), (LiLei,(18,Some(male))))

In [73]:
//rightOuterJoin相当于关系表的右连接
val age = sc.parallelize(Array(("LiLei",18),
                        ("HanMeiMei",16),("Jim",20)))
val gender = sc.parallelize(Array(("LiLei","male"),
                        ("HanMeiMei","female")))
age.rightOuterJoin(gender).collect

age = ParallelCollectionRDD[151] at parallelize at <console>:31
gender = ParallelCollectionRDD[152] at parallelize at <console>:33


Array((HanMeiMei,(Some(16),female)), (LiLei,(Some(18),male)))

**cogroup**

In [75]:
//cogroup相当于对两个输入分别goupByKey然后再对结果进行groupByKey

val x = sc.parallelize(Array(("a",1),("b",2),("a",3)))
val y = sc.parallelize(Array(("a",2),("b",3),("b",5)))

x.cogroup(y).collect.foreach(println)

(a,(CompactBuffer(1, 3),CompactBuffer(2)))
(b,(CompactBuffer(2),CompactBuffer(3, 5)))


x = ParallelCollectionRDD[160] at parallelize at <console>:32
y = ParallelCollectionRDD[161] at parallelize at <console>:33


ParallelCollectionRDD[161] at parallelize at <console>:33

**subtractByKey**

In [76]:
//subtractByKey去除x中那些key也在y中的元素

val x = sc.parallelize(Array(("a",1),("b",2),("c",3)))
val y = sc.parallelize(Array(("a",2),("b",(1,2))))

x.subtractByKey(y).collect

x = ParallelCollectionRDD[164] at parallelize at <console>:32
y = ParallelCollectionRDD[165] at parallelize at <console>:33


Array((c,3))

**foldByKey**

In [77]:
//foldByKey的操作和reduceByKey类似，但是要提供一个初始值
val x = sc.parallelize(Array(("a",1),("b",2),("a",3),("b",5)),1)
x.foldByKey(1)(_*_).collect


x = ParallelCollectionRDD[167] at parallelize at <console>:30


Array((a,3), (b,10))

### 五，持久化操作

如果一个rdd被多个任务用作中间量，那么对其进行cache缓存到内存中对加快计算会非常有帮助。

声明对一个rdd进行cache后，该rdd不会被立即缓存，而是等到它第一次被计算出来时才进行缓存。

可以使用persist明确指定存储级别，常用的存储级别是MEMORY_ONLY和EMORY_AND_DISK。


In [ ]:
//cache缓存到内存中，使用存储级别 MEMORY_ONLY。
//MEMORY_ONLY意味着如果内存存储不下，放弃存储其余部分，需要时重新计算。
val a = sc.parallelize(1 to 10000,5)
a.cache
val sum_a = a.reduce((a,b)=>a+b)
val cnt_a = a.count
val mean_a = sum_a/cnt_a

In [96]:
//persist缓存到内存或磁盘中，默认使用存储级别MEMORY_AND_DISK
//MEMORY_AND_DISK意味着如果内存存储不下，其余部分存储到磁盘中。
//persist可以指定其它存储级别，cache相当于persist(MEMORY_ONLY)
import org.apache.spark.storage.StorageLevel.{MEMORY_AND_DISK}
val a = sc.parallelize(1 to 10000,5)
a.persist(MEMORY_AND_DISK)
val sum_a = a.reduce(_+_)
val cnt_a = a.count
val mean_a = sum_a/cnt_a
a.unpersist()

a = ParallelCollectionRDD[186] at parallelize at <console>:39
sum_a = 50005000
cnt_a = 10000
mean_a = 5000


ParallelCollectionRDD[186] at parallelize at <console>:39

### 六，共享变量

当spark集群在许多节点上运行一个函数时，默认情况下会把这个函数涉及到的对象在每个节点生成一个副本。

但是，有时候需要在不同节点或者节点和Driver之间共享变量。

Spark提供两种类型的共享变量，广播变量和累加器。

广播变量是不可变变量，实现在不同节点不同任务之间共享数据。

广播变量在每个机器上缓存一个只读的变量，而不是为每个task生成一个副本，可以减少数据的传输。

累加器主要是不同节点和Driver之间共享变量，只能实现计数或者累加功能。

累加器的值只有在Driver上是可读的，在节点上不可见。

In [97]:
//广播变量 broadcast 不可变，在所有节点可读

val broads = sc.broadcast(100)

val rdd = sc.parallelize(1 to 10)
rdd.map(_+broads.value).collect

broads.value

broads = Broadcast(157)
rdd = ParallelCollectionRDD[187] at parallelize at <console>:38


100

In [98]:
//累加器 只能在Driver上可读，在其它节点只能进行累加

var total = sc.longAccumulator("total")
val rdd = sc.parallelize(1 to 10,3)

rdd.foreach(x => total.add(x))
total.value

total = LongAccumulator(id: 3803, name: Some(total), value: 55)
rdd = ParallelCollectionRDD[189] at parallelize at <console>:39


55

In [99]:
val rdd = sc.parallelize(for(i<-1 to 100) yield 0.1*i )
val total = sc.doubleAccumulator("sum")
var count = sc.longAccumulator("count")

rdd.foreach(x => {total.add(x)
                  count.add(1)})
total.value/count.value

rdd = ParallelCollectionRDD[190] at parallelize at <console>:37
total = DoubleAccumulator(id: 3829, name: Some(sum), value: 505.0)
count = LongAccumulator(id: 3830, name: Some(count), value: 100)


5.05

### 七，分区操作

分区操作包括改变分区操作，以及针对分区执行的一些转换操作。

coalesce：shuffle可选，默认为false情况下窄依赖，不能增加分区。repartition和partitionBy调用它实现。
repartition：按随机数进行shuffle，相同key不一定在同一个分区
partitionBy：按key进行shuffle，相同key放入同一个分区
HashPartitioner：默认分区器，根据key的hash值进行分区，相同的key进入同一分区，效率较高，key不可为Array.
RangePartitioner：只在排序相关函数中使用，除相同的key进入同一分区，相邻的key也会进入同一分区，key必须可排序。
TaskContext:  获取当前分区id方法 TaskContext.get.partitionId
mapPartitions：每次处理分区内的一批数据，适合需要分批处理数据的情况，比如将数据插入某个表，每批数据只需要开启一次数据库连接，大大减少了连接开支
mapPartitionsWithIndex：类似mapPartitions，提供了分区索引，输入参数为（i，Iterator）
foreachPartition：类似foreach，但每次提供一个Partition的一批数据



**coalesce**

In [101]:
//coalesce 默认shuffle为False，不能增加分区，只能减少分区
//如果要增加分区，要设置shuffle = true
//parallelize等许多操作可以指定分区数
val a = sc.parallelize(1 to 10,3)  
println(a.getNumPartitions)
a.mapPartitions(iter => Iterator(iter.toArray)).collect


3


a = ParallelCollectionRDD[193] at parallelize at <console>:37


Array(Array(1, 2, 3), Array(4, 5, 6), Array(7, 8, 9, 10))

In [102]:
val b = a.coalesce(2) 
b.mapPartitions(iter => Iterator(iter.toArray)).collect()

b = CoalescedRDD[195] at coalesce at <console>:34


Array(Array(1, 2, 3), Array(4, 5, 6, 7, 8, 9, 10))

**repartition**

In [104]:
//repartition按随机数进行shuffle，相同key不一定在一个分区，可以增加分区
//repartition实际上调用coalesce实现，设置了shuffle = true
val a = sc.parallelize(1 to 10,3)  
val c = a.repartition(4) 
c.mapPartitions(iter => Iterator(iter.toArray)).collect()

a = ParallelCollectionRDD[203] at parallelize at <console>:37
c = MapPartitionsRDD[207] at repartition at <console>:38


Array(Array(2, 7), Array(3, 4, 8), Array(5, 9), Array(1, 6, 10))

In [105]:
//repartition按随机数进行shuffle，相同key不一定在一个分区
val a = sc.parallelize(Array(("a",1),("a",1),("a",2),("c",3)))  
val c = a.repartition(2)
c.mapPartitions(iter => Iterator(iter.toArray)).collect()

a = ParallelCollectionRDD[209] at parallelize at <console>:36
c = MapPartitionsRDD[213] at repartition at <console>:37


Array(Array((a,1), (a,2), (c,3)), Array((a,1)))

**partitionBy**

In [107]:
//partitionBy可以指定分区函数为 HashPartitioner或者RangePartitioner.
//HashPartitioner为最常用分区器，根据key的hash值进行分区.
//HashPartitioner相同的key进入同一分区，效率较高，key不可为Array.
//RangePartitioner通常在排序相关场景应用。
//RangePartitioner除相同的key进入同一分区，相邻的key也会进入同一分区。
//RangePartitioner要求key必须可排序。

import org.apache.spark.{HashPartitioner,RangePartitioner}
val a = sc.parallelize(Array(("a",1),("a",1),("b",2),("c",3)))  
val c = a.partitionBy(new HashPartitioner(2))
c.mapPartitions(iter => Iterator(iter.toArray)).collect()


a = ParallelCollectionRDD[218] at parallelize at <console>:44
c = ShuffledRDD[219] at partitionBy at <console>:45


Array(Array((b,2)), Array((a,1), (a,1), (c,3)))

**mapPartitions**

In [108]:
//mapPartitions可以对每个分区分别执行操作
//每次处理分区内的一批数据，适合需要按批处理数据的情况
//例如将数据写入数据库时，可以极大的减少连接次数。
//mapPartitions的输入分区内数据组成的Iterator，其输出也需要是一个Iterator
//以下例子查看每个分区内的数据
val a = sc.parallelize(1 to 10,2)
a.mapPartitions(iter => Iterator(iter.toArray)).collect()

a = ParallelCollectionRDD[221] at parallelize at <console>:41


Array(Array(1, 2, 3, 4, 5), Array(6, 7, 8, 9, 10))

**mapPartitionsWithIndex**

In [109]:
//mapPartitionsWithIndex可以获取两个参数
//即分区id和每个分区内的数据组成的Iterator
val a = sc.parallelize(1 to 10,2)
var b = a.mapPartitionsWithIndex{
        (pid,iter) => {
          var result = List[String]()
            var i = 0
            while(iter.hasNext){
              i += iter.next()
            }
            result.::(pid + "|" + i).iterator
           
        }
      }
b.collect

a = ParallelCollectionRDD[223] at parallelize at <console>:40
b = MapPartitionsRDD[224] at mapPartitionsWithIndex at <console>:41


Array(0|15, 1|40)

In [111]:
//利用TaskContext可以获取当前每个元素的分区
import org.apache.spark.TaskContext

val a = sc.parallelize(1 to 5,3)
val c = a.map((TaskContext.get.partitionId,_))
c.collect

a = ParallelCollectionRDD[227] at parallelize at <console>:41
c = MapPartitionsRDD[228] at map at <console>:42


Array((0,1), (1,2), (1,3), (2,4), (2,5))

**foreachPartitions**

In [112]:
//foreachPartition对每个分区分别执行操作
//范例：求每个分区内最大值的和
var total = sc.doubleAccumulator("total")

val a = sc.parallelize(1 to 100,3)

a.foreachPartition(iter =>{
    val m = iter.reduce((x,y) => if(x>y) x else y)
    total.add(m)
})

total.value

total = DoubleAccumulator(id: 4281, name: Some(total), value: 199.0)
a = ParallelCollectionRDD[229] at parallelize at <console>:48


199.0

**aggregate** 

In [114]:
//aggregate是一个Action操作
//aggregate比较复杂，先对每个分区执行一个函数，再对每个分区结果执行一个合并函数。
//例子：求元素之和以及元素个数
//三个参数，第一个参数为初始值，第二个为分区执行函数，第三个为结果合并执行函数。
val rdd = sc.parallelize(1 to 20,3)
rdd.aggregate((0,0))((t,x)=>{println(t)
                             (t._1+x,t._2+1)},
                     (p,q)=>{println(p,q)
                         (p._1+q._1,p._2+q._2)})

rdd = ParallelCollectionRDD[230] at parallelize at <console>:44


(210,20)

**aggregateByKey**

In [115]:
//aggregateByKey的操作和aggregate类似，但是会对每个key分别进行操作
//第一个参数为初始值，第二个参数为分区内归并函数，第三个参数为分区间归并函数

val a = sc.parallelize(Array(("a",1),("b",1),("c",2),
                             ("a",2),("b",3)),3)

val b = a.aggregateByKey(0)((a,b)=>if(a>b) a else b,
                            (a,b)=>if(a>b) a else b)

b.collect

a = ParallelCollectionRDD[231] at parallelize at <console>:43
b = ShuffledRDD[232] at aggregateByKey at <console>:46


Array((c,2), (a,2), (b,3))